In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import xesmf as xe
from metpy.units import units
from metpy import calc as mpcalc
import netCDF4
from netCDF4 import Dataset
from datetime import datetime, date, timedelta

In [2]:
#Opening Climo data from ERA-I 
#ct_ds = xr.open_dataset("/erai/climo/mean/t.mean.climo.nc")
ct_rgrd_ds = xr.open_dataset("/home11/grad/2021/gd673524/research/gdines/data/realtime/ECMWF/ECMWF_ct_rgrd.nc")

stdv_t_ds = xr.open_dataset('/erai/climo/stdv/t.stdv.climo.nc')
stdv_t_rgrd_ds = xr.open_dataset('/home11/grad/2021/gd673524/research/gdines/data/realtime/ECMWF/ct_stdv_rgrd.nc')

#Reading in Datasets (odd=hightop, even=lowtop)
t_ds = xr.open_dataset("/home11/grad/2021/gd673524/research/gdines/data/realtime/ECMWF/RT_ECMWF_t.nc")
t_2ds = xr.open_dataset("/home11/grad/2021/gd673524/research/gdines/data/realtime/ECCC/RT_ECCC_t.nc")
t_3ds = xr.open_dataset("/home11/grad/2021/gd673524/research/gdines/data/realtime/UKMO/RT_UKMO_t.nc")
t_4ds = xr.open_dataset("/home11/grad/2021/gd673524/research/gdines/data/realtime/CMA/RT_CMA_t.nc")
t_5ds = xr.open_dataset("/home11/grad/2021/gd673524/research/gdines/data/realtime/JMA/RT_JMA_t.nc")
t_6ds = xr.open_dataset("/home11/grad/2021/gd673524/research/gdines/data/realtime/CNR/RT_CNR_t.nc")

In [4]:
#Regridding ERAI data using xesmf
#stdv_ds_out = xr.Dataset(
#    {
#        "lat": (["lat"], np.arange(-90, 91.5, 1.5)),
#        "lon": (["lon"], np.arange(0, 360, 1.5)),
#    }
#)

#regridder = xe.Regridder(stdv_t_ds, stdv_ds_out, "bilinear")
#regridder

In [5]:
#climo_stdv_t_rgrd = regridder(stdv_t_ds)
#climo_stdv_t_rgrd

In [6]:
#Making regridded data into a netCDF file
#stdv_t_rgrd_ds = climo_stdv_t_rgrd.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/ECMWF/ct_stdv_rgrd.nc",
#                               mode='w')

In [7]:
### Vertical level specificaton
vlevel = 1000
levelStr = str(vlevel)

#Hour Specification
Hour = 0
hourStr = str(Hour)

#Box coordinates:
#Whole central US box =  [[48,-108], [48,-87], [33,-87], [33,-108],[48,-108]]
#ConCan = lat min/max:25/75, lon min/max:200/300
lat_min = 25
lat_max = 75
lon_min = 200
lon_max = 300

In [8]:
#Reducing the Model datasets
t = t_ds['t'].sel(level=vlevel)#<--forecasted temps at 1000mb
t_rd = t.sel(latitude=slice(lat_max,lat_min), longitude=slice(lon_min,lon_max))

t2 = t_2ds['t'].sel(level=vlevel)
t2_rd = t2.sel(latitude=slice(lat_max,lat_min), longitude=slice(lon_min,lon_max))

t3 = t_3ds['t'].sel(level=vlevel)
t3_rd = t3.sel(latitude=slice(lat_max,lat_min), longitude=slice(lon_min,lon_max))

t4 = t_4ds['t'].sel(level=vlevel)
t4_rd = t4.sel(latitude=slice(lat_max,lat_min), longitude=slice(lon_min,lon_max))

t5 = t_5ds['t'].sel(level=vlevel)
t5_rd = t5.sel(latitude=slice(lat_max,lat_min), longitude=slice(lon_min,lon_max))

t6 = t_6ds['t'].sel(level=vlevel)
t6_rd = t6.sel(latitude=slice(lat_max,lat_min), longitude=slice(lon_min,lon_max))

In [9]:
#Reducing the Climatology dataset
#ct_rgrd_ds1 = ct_rgrd_ds.reindex(lat=list(reversed(ct_rgrd_ds.lat)))
ct_rd0 = ct_rgrd_ds['t'].sel(lev=vlevel, hour=hourStr,) #<-- this is climo at 1000mb for only 00z
ct_rd = ct_rd0.sel(lat=slice(lat_min,lat_max), lon=slice(lon_min,lon_max))
ct_rd_ds = ct_rd.reindex(lat=list(reversed(ct_rd.lat)))

stdvt_rd0 = stdv_t_rgrd_ds['t'].sel(lev=vlevel, hour=hourStr,) #<-- this is climo at 1000mb for only 00z
stdvt_rd = stdvt_rd0.sel(lat=slice(lat_min,lat_max), lon=slice(lon_min,lon_max))
stdvt_rd_ds = stdvt_rd.reindex(lat=list(reversed(stdvt_rd.lat)))

In [10]:
t6_rd

<xarray.DataArray 't' (date: 4, step: 33, number: 40, latitude: 34,
                       longitude: 67)>
[12027840 values with dtype=float32]
Coordinates:
  * longitude  (longitude) float32 201.0 202.5 204.0 205.5 ... 297.0 298.5 300.0
  * latitude   (latitude) float32 75.0 73.5 72.0 70.5 ... 30.0 28.5 27.0 25.5
    level      int32 1000
  * number     (number) int32 1 2 3 4 5 6 7 8 9 ... 32 33 34 35 36 37 38 39 40
  * step       (step) timedelta64[ns] 0 days 1 days 2 days ... 31 days 32 days
  * date       (date) datetime64[ns] 2021-01-07 2021-01-14 2021-01-21 2021-01-28
Attributes:
    units:          K
    long_name:      Temperature
    standard_name:  air_temperature

In [10]:
stdvt_rd_ds

<xarray.DataArray 't' (time: 366, lat: 34, lon: 67)>
[833748 values with dtype=float64]
Coordinates:
  * time     (time) float64 1.0 2.0 3.0 4.0 5.0 ... 363.0 364.0 365.0 366.0
  * lon      (lon) float64 201.0 202.5 204.0 205.5 ... 295.5 297.0 298.5 300.0
  * lat      (lat) float64 75.0 73.5 72.0 70.5 69.0 ... 31.5 30.0 28.5 27.0 25.5
    hour     float64 0.0
    lev      int32 1000

In [ ]:
#Calculating ensemble means
t_ensmn = t_rd.mean(dim='number')
t2_ensmn = t2_rd.mean(dim='number')
t3_ensmn = t3_rd.mean(dim='number')
t4_ensmn = t4_rd.mean(dim='number')
t5_ensmn = t5_rd.mean(dim='number')
t6_ensmn = t6_rd.mean(dim='number')

In [11]:
#Calculating the area mean temperature
ct_llmn = ct_rd_ds.mean(dim=['lat','lon'])
stdvt_llmn = stdvt_rd_ds.mean(dim=['lat', 'lon'])
#t_llmn = t_rd.mean(dim=['latitude', 'longitude'])
#t2_llmn = t2_rd.mean(dim=['latitude', 'longitude'])
#t3_llmn = t3_rd.mean(dim=['latitude', 'longitude'])
#t4_llmn = t4_rd.mean(dim=['latitude', 'longitude'])
#t5_llmn = t5_rd.mean(dim=['latitude', 'longitude'])
#t6_llmn = t6_rd.mean(dim=['latitude', 'longitude'])

In [12]:
stdvt_rd_ds

<xarray.DataArray 't' (time: 366, lat: 34, lon: 67)>
array([[[5.647723, 5.624033, ..., 7.071272, 7.172698],
        [5.969191, 5.96447 , ..., 7.1912  , 7.597021],
        ...,
        [2.058244, 2.019987, ..., 1.619026, 1.591935],
        [1.81691 , 1.785241, ..., 1.406594, 1.370857]],

       [[5.66572 , 5.643042, ..., 7.092618, 7.196642],
        [5.991998, 5.988392, ..., 7.19108 , 7.605042],
        ...,
        [2.064817, 2.025945, ..., 1.624593, 1.597824],
        [1.824041, 1.791821, ..., 1.410259, 1.374656]],

       ...,

       [[5.612454, 5.586655, ..., 7.029753, 7.125372],
        [5.92563 , 5.918382, ..., 7.193131, 7.580494],
        ...,
        [2.044317, 2.007443, ..., 1.607473, 1.5796  ],
        [1.802005, 1.77156 , ..., 1.398907, 1.362868]],

       [[5.629969, 5.605238, ..., 7.050322, 7.148948],
        [5.947053, 5.941119, ..., 7.191909, 7.588864],
        ...,
        [2.051408, 2.013818, ..., 1.613319, 1.585859],
        [1.809562, 1.778487, ..., 1.402811, 1.366928]]])
Coordinates:
  * time     (time) float64 1.0 2.0 3.0 4.0 5.0 ... 363.0 364.0 365.0 366.0
  * lon      (lon) float64 201.0 202.5 204.0 205.5 ... 295.5 297.0 298.5 300.0
  * lat      (lat) float64 75.0 73.5 72.0 70.5 69.0 ... 31.5 30.0 28.5 27.0 25.5
    hour     float64 0.0
    lev      int32 1000

In [13]:
#Writing reduced datasets to NetCDFs
#ct_rd_ds = ct_rd_ds.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/rd_t/ct_rd.nc",
#                               mode='w')
stdvt_rd_ds = stdvt_rd_ds.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/box_proj/CONCAN_data/rd_t/stdvt_rd.nc",
                               mode='w')
#t_rd_ds = t_rd.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/rd_t/ECMWF_t_rd.nc",
#                             mode='w')
#t2_rd_ds = t2_rd.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/rd_t/ECCC_t_rd.nc",
#                            mode='w')
#t3_rd_ds = t3_rd.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/rd_t/UKMO_t_rd.nc",
#                            mode='w')
#t4_rd_ds = t4_rd.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/rd_t/CMA_t_rd.nc",
#                            mode='w')
#t5_rd_ds = t5_rd.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/rd_t/JMA_t_rd.nc",
#                            mode='w')
#t6_rd_ds = t6_rd.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/rd_t/CNR_t_rd.nc",
#                            mode='w')

In [ ]:
#Writing Ensemble Mean temps to NetCDFs
t_ensmn_ds = t_ensmn.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/ensmn_t/ECMWF_ensmn_t.nc",
                             mode='w')
t2_ensmn_ds = t2_ensmn.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/ensmn_t/ECCC_ensmn_t.nc",
                            mode='w')
t3_ensmn_ds = t3_ensmn.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/ensmn_t/UKMO_ensmn_t.nc",
                            mode='w')
t4_ensmn_ds = t4_ensmn.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/ensmn_t/CMA_ensmn_t.nc",
                            mode='w')
t5_ensmn_ds = t5_ensmn.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/ensmn_t/JMA_ensmn_t.nc",
                            mode='w')
t6_ensmn_ds = t6_ensmn.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/ensmn_t/CNR_ensmn_t.nc",
                            mode='w')

In [14]:
#Writing mean area temps to NetCDFs
#ct_llmn_ds = ct_llmn.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/llmn_t/ct_llmn_t.nc",
#                            mode='w')
stdvt_llmn_ds = stdvt_llmn.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/box_proj/CONCAN_data/llmn_t/stdvt_llmn_t.nc",
                            mode='w')
#t_llmn_ds = t_llmn.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/llmn_t/ECMWF_llmn_t.nc",
#                            mode='w')
#t2_llmn_ds = t2_llmn.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/llmn_t/ECCC_llmn_t.nc",
#                            mode='w')
#t3_llmn_ds = t3_llmn.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/llmn_t/UKMO_llmn_t.nc",
#                            mode='w')
#t4_llmn_ds = t4_llmn.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/llmn_t/CMA_llmn_t.nc",
#                            mode='w')
#t5_llmn_ds = t5_llmn.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/llmn_t/JMA_llmn_t.nc",
#                            mode='w')
#t6_llmn_ds = t6_llmn.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/llmn_t/CNR_llmn_t.nc",
#                            mode='w')